# Enviorment setup

## Libraries

In [3]:
import duckdb as db
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import sys

from importlib import reload
from pathlib import Path


In [4]:
# Project path
project_path = Path('.').resolve().parent

# Include in path
sys.path.append(str(project_path))

In [5]:
from src import auxiliary as aux
reload(aux)

<module 'src.auxiliary' from '/home/henrique/Projetos/e-commerce-fraud-detection/src/auxiliary.py'>

## Parameters

In [6]:
prm = aux.load_parameters()

## Plotly

In [7]:
# Set Plotly theme and render
pio.templates.default = prm['plotly']['theme']
pio.renderers.default = prm['plotly']['renderer']

# EDA

## Data

### Load

In [22]:
# Load .csv file
purchases = pd.read_csv(project_path.joinpath('data', 'raw', 'purchase-database.csv'))

# Rename columns (pt-BR to en-US)
purchases = purchases.rename(columns=prm['purchase-database']['columns'])

# Reorder columns
purchases = purchases[prm['purchase-database']['columns'].values()]

purchases.head().T

,0,1,2,3,4
product_name,Máquininha Corta Barba Cabelo Peito Perna Pelo...,Avental Descartavel Manga Longa - 50 Un. Tnt ...,Bicicleta Mountain Fire Bird Rodado 29 Alumini...,Caneta Delineador Carimbo Olho Gatinho Longo 2...,Resident Evil Operation Raccoon City Ps3
product_category,cat_8d714cd,cat_64b574b,cat_e9110c5,cat_d06e653,cat_6c4cfdc
purchase_date,2020-03-27 11:51:16,2020-04-15 19:58:08,2020-03-25 18:13:38,2020-04-16 16:03:10,2020-04-02 10:24:45
purchase_value,5.64,124.71,339.32,3.54,3.53
country,BR,BR,AR,BR,BR
score_1,4,4,4,4,2
score_2,0.7685,0.755,0.7455,0.7631,0.7315
score_3,94436.24,9258.5,242549.09,18923.9,5728.68
score_4,20.0,1.0,3.0,50.0,15.0
score_5,0.444828,0.0,0.0,0.482385,0.0


### NaN check

In [9]:
# % of NaN values in each column
100 * purchases.isna().sum() / purchases.shape[0]

product_name            0.000000
product_category        0.000000
purchase_date           0.000000
purchase_value          0.000000
country                 0.129333
score_1                 0.000000
score_2                 8.656000
score_3                 8.656000
score_4                 0.243333
score_5                 0.000000
score_6                 0.007333
score_7                 0.000000
score_8                 0.000000
score_9                 0.007333
score_10                0.243333
document_delivery_1     0.000000
document_delivery_2    72.571333
document_delivery_3     0.000000
fraud_score_model       0.000000
fraud                   0.000000
dtype: float64

For document delivery fields, a missing information is interpreted as a "no" answer.

### Description of numerical fields

In [13]:
purchases.describe()

,purchase_value,score_1,score_2,score_3,score_4,score_5,score_6,score_7,score_8,score_9,score_10,document_delivery_1,fraud_score_model,fraud
count,150000.000000,150000.000000,137016.000000,1.370160e+05,149635.000000,150000.000000,149989.000000,150000.000000,150000.000000,149989.000000,149635.000000,150000.000000,150000.000000,150000.000000
mean,43.523134,3.705407,0.728115,2.604451e+05,21.677669,0.220641,51.169352,14.193513,0.497532,2305.409403,299.969579,0.902353,48.066240,0.050000
std,91.557888,0.753206,0.132943,8.464361e+05,20.062146,2.434995,709.472904,14.161216,0.288348,1712.379601,321.075806,0.296837,28.995122,0.217946
min,0.020000,1.000000,0.000000,1.600000e-01,0.000000,0.000000,-5.000000,0.000000,0.000004,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9.380000,4.000000,0.678400,9.679915e+03,2.000000,0.000000,1.000000,3.000000,0.246819,910.000000,42.000000,1.000000,23.000000,0.000000
50%,20.610000,4.000000,0.755500,4.371165e+04,14.000000,0.104875,8.000000,9.000000,0.495990,1937.000000,193.000000,1.000000,48.000000,0.000000
75%,40.692500,4.000000,0.806500,1.454436e+05,50.000000,0.282938,33.000000,21.000000,0.746508,3445.000000,459.000000,1.000000,73.000000,0.000000
max,3696.350000,4.000000,1.000000,1.387874e+07,50.000000,833.333333,145274.000000,58.000000,0.999995,7544.000000,2225.000000,1.000000,100.000000,1.000000


### Description of categorical fields

In [17]:
# Select all columns with non numerical fields
non_numerical_columns = purchases.select_dtypes(exclude='number').columns

for column in non_numerical_columns:
    print(f'{column}: {len(purchases[column].unique())} unique values')
    display(purchases[column].value_counts())
    print('\n')

product_name: 127804 unique values


product_name
Fone De Ouvido Sem Fio Xiaomi Airdots Preto                     83
Xiaomi Redmi Note 8 Dual Sim 64 Gb Azul-netuno 4 Gb Ram         51
Clean Sept - Álcool Em Gel 70% 500 Ml                           49
Tapete Ilustrativo Pista Carrinho Hot Wheels 0,60x1,20m         47
Máscara Descartável Tripla Com Elástico Proteção Anvisa 50un    47
                                                                ..
Interruptor Lâmpada 3 Vias Canais Controle Wirells 110v/220v     1
Fone De Ouvido Para iPad iPhone 4s 5 5c 5s 6 6s Plus+ Brinde     1
2 Potes Moder Diet 500mg 100caps Total 200caps                   1
Pacote Master Revit Diamante + Extras + Brindes 2017 2020        1
Copita Menstrual Silicona Médica Importada Aneer                 1
Name: count, Length: 127804, dtype: int64



product_category: 8324 unique values


product_category
cat_43b9c10    2331
cat_d69bed9    1799
cat_604df77    1784
cat_6c4cfdc    1449
cat_4249bd8    1191
               ... 
cat_d6c71e0       1
cat_34ac07c       1
cat_4858280       1
cat_be3757f       1
cat_63704c0       1
Name: count, Length: 8324, dtype: int64



purchase_date: 145813 unique values


...

country
BR    111628
AR     31964
UY      2967
US      2273
SE       358
MX       236
RU        73
ES        69
CO        64
GB        43
FR        18
IT        14
PT        13
CL         9
DE         9
PE         5
PY         4
EC         4
AU         3
UA         3
CN         3
CA         3
BE         3
NZ         3
CH         3
PA         3
BO         2
LB         2
PR         2
NL         2
CR         2
AE         2
AD         1
GE         1
KR         1
IN         1
HN         1
NI         1
GR         1
DO         1
EG         1
TR         1
BS         1
PH         1
GH         1
TW         1
FI         1
NO         1
IL         1
JP         1
ZA         1
Name: count, dtype: int64



document_delivery_2: 3 unique values


document_delivery_2
Y    24091
N    17052
Name: count, dtype: int64



document_delivery_3: 2 unique values


document_delivery_3
Y    83129
N    66871
Name: count, dtype: int64

### Transformations

In [30]:
# Convert 'purchase_date' to datetime
purchases['purchase_date'] = pd.to_datetime(purchases['purchase_date'], yearfirst=True)

purchases['purchase_date']

0        2020-03-27 11:51:16
1        2020-04-15 19:58:08
2        2020-03-25 18:13:38
3        2020-04-16 16:03:10
4        2020-04-02 10:24:45
                 ...        
149995   2020-04-11 19:31:07
149996   2020-03-11 20:21:35
149997   2020-03-11 19:36:07
149998   2020-03-16 07:13:24
149999   2020-03-26 08:47:12
Name: purchase_date, Length: 150000, dtype: datetime64[ns]

In [32]:
# All 3 document deliveries to 1 and 0 binaries
purchases['document_delivery_2'] = purchases['document_delivery_2'].map({'Y': 1, 'N': 0})
purchases['document_delivery_2'] = purchases['document_delivery_2'].fillna(0).astype(int)

purchases['document_delivery_3'] = purchases['document_delivery_3'].map({'Y': 1, 'N': 0})

In [ ]:
purchases

,product_name,product_category,purchase_date,purchase_value,country,score_1,score_2,score_3,score_4,score_5,score_6,score_7,score_8,score_9,score_10,document_delivery_1,document_delivery_2,document_delivery_3,fraud_score_model,fraud
0,Máquininha Corta Barba Cabelo Peito Perna Pelo...,cat_8d714cd,2020-03-27 11:51:16,5.64,BR,4,0.7685,94436.24,20.0,0.444828,1.0,5,0.883598,240.0,102.0,1,0,0,66,0
1,Avental Descartavel Manga Longa - 50 Un. Tnt ...,cat_64b574b,2020-04-15 19:58:08,124.71,BR,4,0.7550,9258.50,1.0,0.000000,33.0,0,0.376019,4008.0,0.0,1,1,0,72,0
2,Bicicleta Mountain Fire Bird Rodado 29 Alumini...,cat_e9110c5,2020-03-25 18:13:38,339.32,AR,4,0.7455,242549.09,3.0,0.000000,19.0,23,0.516368,1779.0,77.0,1,0,0,95,0
3,Caneta Delineador Carimbo Olho Gatinho Longo 2...,cat_d06e653,2020-04-16 16:03:10,3.54,BR,4,0.7631,18923.90,50.0,0.482385,18.0,23,0.154036,1704.0,1147.0,1,0,1,2,0
4,Resident Evil Operation Raccoon City Ps3,cat_6c4cfdc,2020-04-02 10:24:45,3.53,BR,2,0.7315,5728.68,15.0,0.000000,1.0,2,0.855798,1025.0,150.0,1,0,0,76,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149995,Aparelho Lipocavitação Ultrassônico + Gel Cond...,cat_a5b2091,2020-04-11 19:31:07,47.15,BR,4,0.8191,21393.63,7.0,0.000000,1.0,4,0.808366,2009.0,434.0,1,0,0,95,0
149996,"Sellos De Goma Automaticos, Personalizados.",cat_e39ab7e,2020-03-11 20:21:35,9.69,UY,4,NaN,NaN,24.0,0.384615,11.0,20,0.989981,499.0,135.0,1,0,1,0,0
149997,Hélice 3 Pás Alumínio Rabeta 6.5 Hp Pesca Barc...,cat_ee6ecc8,2020-03-11 19:36:07,5.97,BR,4,NaN,NaN,3.0,0.477778,1.0,19,0.763939,127.0,127.0,1,0,1,15,0
149998,Tela Display Lcd Galaxy J7 Neo J701 Com Brilho...,cat_237e2d0,2020-03-16 07:13:24,25.83,BR,4,0.6067,152906.86,1.0,0.099175,133.0,3,0.382728,4373.0,123.0,1,0,1,59,0


### Check purchase_date distribution

In [35]:
purchase_dates = purchases[['purchase_date']].copy()
purchase_dates['year_month'] = purchase_dates['purchase_date'].dt.to_period('M')
purchase_dates[['year_month']].groupby('year_month').size()

year_month
2020-03    76961
2020-04    73039
Freq: M, dtype: int64

All the purchases in database occurred in two months of 2020 year, so we'll not consider this field in the model.

Futhermore, we don't have the timezone, so we'll not consider the hour of the day of purchase in the model neither.

## Visualizations